In [ ]:
import os
import os.path
import re
import sys
import codecs
import hashlib
import json
import os.path
import shutil
import uuid
import skimage.draw
import skimage.color
import skimage.io


from pathlib import Path
import json

analysis_root_dir = "C:/Users/zhuhuiling/Desktop/scratch1"
path = Path(analysis_root_dir)
all_json_file = list(path.glob('**/*.json'))
parse_result = []
for json_file in all_json_file:
    # 获取所在目录的名称
    service_name = json_file.parent.stem
    with json_file.open() as f:
        json_result = json.load(f)
    parse_result.append(json_result) 


r,c=(224,224)
groups = ("training2", "test2")
for group in groups:
    dictionaries = [] 
    for i in parse_result[0:20]: 
        pathname = "im/"+ i["imagePath"]
        if os.path.exists(path):
            dictionary = {
                        "image": {
                        "pathname": pathname,
                        "shape": {
                                "r": r,
                                "c": c,
                                "channels": 3
                            }
                        },
                        "objects": []
                    }
            for j in i["shapes"]:
                category = j['label']
                (bounding_box_r, bounding_box_c)=j['points']
                maximum_r=max(bounding_box_r)  
                minimum_r= min(bounding_box_r)
                minimum_c = min(bounding_box_c)
                maximum_c= max(bounding_box_c)
                object_dictionary = {
                        "bounding_box": {
                             "minimum": {
                                 "r": minimum_r - 1,
                                 "c": minimum_c - 1
                         },
                         "maximum": {
                                "r": maximum_r - 1,
                                "c": maximum_c - 1
                            }
                       },
                        "category": category
                    }
                dictionary["objects"].append(object_dictionary)
            dictionaries.append(dictionary)
            filename = "{}.json".format(group)
            with open(filename, "w") as stream:
                json.dump(dictionaries, stream)#dumps是将dict转化成str格式，loads是将str转化成dict格式。


#得到training1.json和test.json        
        

In [1]:
"""
A simple example for ploting two figures of a exponential
function in order to test the autonomy of the gallery
stacking multiple images.
"""

import keras
import keras_rcnn
import keras_rcnn.datasets.shape 
import keras_rcnn.preprocessing
import keras_rcnn.models


import json

import jsonschema

import pkg_resources


def load_data():
    resource_path = "/".join(["data1", "schema.json"])

    with open(pkg_resources.resource_filename("keras_rcnn", resource_path)) as stream:
        schema = json.load(stream)

    resource_path = "/".join(["data1", "shape", "training2.json"])

    with open(pkg_resources.resource_filename("keras_rcnn", resource_path)) as stream:
        training_dictionary = json.load(stream)

    jsonschema.validate(training_dictionary, schema)

    for dictionary in training_dictionary:
        resource_path = "/".join(["data1", "shape", dictionary["image"]["pathname"]])
        pathname = pkg_resources.resource_filename("keras_rcnn", resource_path)
        dictionary["image"]["pathname"] = pathname

    resource_path = "/".join(["data1", "shape", "test2.json"])

    with open(pkg_resources.resource_filename("keras_rcnn", resource_path)) as stream:
        test_dictionary = json.load(stream)

    jsonschema.validate(test_dictionary, schema)

    for dictionary in test_dictionary:
        resource_path = "/".join(["data1", "shape", dictionary["image"]["pathname"]])
        pathname = pkg_resources.resource_filename("keras_rcnn", resource_path)
        dictionary["image"]["pathname"] = pathname

    return training_dictionary, test_dictionary



D:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:

training_dictionary, test_dictionary = load_data()
training_dictionary

#开始训练模型
categories = {"scratch":1}

generator = keras_rcnn.preprocessing.ObjectDetectionGenerator()

generator = generator.flow_from_dictionary(
        dictionary=training_dictionary,
        categories=categories,
        target_size=(224, 224)
    )
validation_data = keras_rcnn.preprocessing.ObjectDetectionGenerator()
validation_data = validation_data.flow_from_dictionary(
        dictionary=test_dictionary,
        categories=categories,
        target_size=(224, 224)
    )
validation_data
keras.backend.set_learning_phase(1)
model = keras_rcnn.models.RCNN(
        categories = {"scratch":1},
        dense_units=512,
        input_shape=(224, 224, 3)
    )


optimizer = keras.optimizers.Adam()
model.compile(optimizer)

model.fit_generator(
        epochs=3,
        generator=generator,
        validation_data=validation_data
    )


D:\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3


D:\Anaconda3\lib\site-packages\skimage\transform\_warps.py:24: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '
D:\Anaconda3\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
D:\Anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


20/20 [==============================] - 110s 5s/step - loss: 189.7758 - val_loss: 13.1474
Epoch 2/3
20/20 [==============================] - 92s 5s/step - loss: 12.8258 - val_loss: 9.3401
Epoch 3/3
20/20 [==============================] - 91s 5s/step - loss: 9.8792 - val_loss: 8.0751
